In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

In [ ]:
RATED_FOOD_PATH = '/content/drive/My Drive/capstone_rec_sys/Tensorflow Rec System/Food and Rating Data/Exported Rated Food 07-06-2021 16-35-04.csv'

**1. Load Healthy FnB Data**

In [ ]:
df = pd.read_csv(RATED_FOOD_PATH)
df.head()

,food_id,food_code,name,category,type,calories,protein,carbs,fat,fiber,sugar,vitamin_a,vitamin_b6,vitamin_b12,vitamin_c,vitamin_d,vitamin_e
0,8,11432000,"Yogurt, low fat milk, fruit",Yogurt,Food,89.0,4.66,14.46,1.37,0.1,12.01,12.0,0.046,0.49,1.8,1.2,0.04
1,13,11435100,"Yogurt, Greek, with oats",Yogurt,Food,160.0,8.90,22.38,4.29,1.9,12.07,88.0,0.077,0.53,0.1,0.7,0.24
2,15,11446000,"Yogurt parfait, low fat, with fruit",Yogurt,Food,84.0,3.36,15.86,1.01,1.1,11.68,30.0,0.233,0.88,14.3,1.2,0.25
3,16,14710200,"Beer cheese soup, made with milk",Soups,Food,192.0,6.19,6.52,15.43,0.3,0.80,182.0,0.037,0.26,0.3,0.3,0.45
4,23,26109133,"Cod, coated, baked or broiled, no added fat",Fish,Food,137.0,17.32,13.10,1.16,0.6,1.15,9.0,0.121,1.81,0.0,0.6,0.54


In [ ]:
knn_df = df.copy()
knn_df.drop(['food_id','food_code','name','category','type'],axis =1 ,inplace = True)
knn_df.head()

,calories,protein,carbs,fat,fiber,sugar,vitamin_a,vitamin_b6,vitamin_b12,vitamin_c,vitamin_d,vitamin_e
0,89.0,4.66,14.46,1.37,0.1,12.01,12.0,0.046,0.49,1.8,1.2,0.04
1,160.0,8.90,22.38,4.29,1.9,12.07,88.0,0.077,0.53,0.1,0.7,0.24
2,84.0,3.36,15.86,1.01,1.1,11.68,30.0,0.233,0.88,14.3,1.2,0.25
3,192.0,6.19,6.52,15.43,0.3,0.80,182.0,0.037,0.26,0.3,0.3,0.45
4,137.0,17.32,13.10,1.16,0.6,1.15,9.0,0.121,1.81,0.0,0.6,0.54


**2. Scale Dataframe using MinMaxScaller()**

In [ ]:
min_max_scaler = MinMaxScaler()
knn_df = min_max_scaler.fit_transform(knn_df)
knn_df

array([[0.39732143, 0.15164335, 0.60049834, ..., 0.036     , 0.08759124,
        0.00873362],
       [0.71428571, 0.28961926, 0.92940199, ..., 0.002     , 0.05109489,
        0.05240175],
       [0.375     , 0.10933941, 0.65863787, ..., 0.286     , 0.08759124,
        0.05458515],
       ...,
       [0.27232143, 0.21444842, 0.035299  , ..., 0.388     , 0.08029197,
        1.        ],
       [0.02232143, 0.        , 0.05730897, ..., 0.        , 0.        ,
        0.        ],
       [0.02232143, 0.        , 0.05730897, ..., 0.        , 0.        ,
        0.        ]])

**3. Build the Model**

In [ ]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(knn_df)
distances, indices = nbrs.kneighbors(knn_df)


all_code = list(df['food_code'].values)

In [ ]:
df.head()

,food_id,food_code,name,category,type,calories,protein,carbs,fat,fiber,sugar,vitamin_a,vitamin_b6,vitamin_b12,vitamin_c,vitamin_d,vitamin_e
0,8,11432000,"Yogurt, low fat milk, fruit",Yogurt,Food,89.0,4.66,14.46,1.37,0.1,12.01,12.0,0.046,0.49,1.8,1.2,0.04
1,13,11435100,"Yogurt, Greek, with oats",Yogurt,Food,160.0,8.90,22.38,4.29,1.9,12.07,88.0,0.077,0.53,0.1,0.7,0.24
2,15,11446000,"Yogurt parfait, low fat, with fruit",Yogurt,Food,84.0,3.36,15.86,1.01,1.1,11.68,30.0,0.233,0.88,14.3,1.2,0.25
3,16,14710200,"Beer cheese soup, made with milk",Soups,Food,192.0,6.19,6.52,15.43,0.3,0.80,182.0,0.037,0.26,0.3,0.3,0.45
4,23,26109133,"Cod, coated, baked or broiled, no added fat",Fish,Food,137.0,17.32,13.10,1.16,0.6,1.15,9.0,0.121,1.81,0.0,0.6,0.54


**4. Try to find 10 similar food to the target based on the nutrition**

In [ ]:
searched_food = 92510955
searched_food_name = df.loc[df['food_code']==searched_food].head(1).name.tolist()
searched_food_code = all_code.index(searched_food)

In [ ]:
print("FnB that related to : ",searched_food_name[0])
df.loc[indices[searched_food_code]]

FnB that related to :  Lemonade, fruit juice drink


,food_id,food_code,name,category,type,calories,protein,carbs,fat,fiber,sugar,vitamin_a,vitamin_b6,vitamin_b12,vitamin_c,vitamin_d,vitamin_e
126,416,92510955,"Lemonade, fruit juice drink",Fruit drinks,Beverage,46.0,0.00,12.08,0.00,0.0,11.67,0.0,0.004,0.00,3.1,0.0,0.00
112,379,64101010,Apple cider,Fruit drinks,Beverage,46.0,0.10,11.30,0.13,0.2,9.62,0.0,0.018,0.00,0.9,0.0,0.01
106,358,11519205,"Strawberry milk, fat free",Milk,Beverage,61.0,3.18,12.03,0.09,0.0,11.90,59.0,0.054,0.54,0.0,1.0,0.00
113,380,64104010,"Apple juice, 100%",Fruit drinks,Beverage,46.0,0.10,11.30,0.13,0.2,9.62,0.0,0.018,0.00,10.3,0.0,0.01
103,333,11511400,"Chocolate milk, ready to drink, low fat",Milk,Beverage,64.0,3.46,10.15,1.11,0.1,9.94,52.0,0.047,0.23,0.4,1.1,0.00
108,366,42404010,"Coconut water, sweetened",Fruit drinks,Beverage,37.0,0.21,9.07,0.02,0.0,8.78,0.0,0.000,0.00,9.4,0.0,0.00
102,332,11511300,"Chocolate milk, ready to drink, fat free",Milk,Beverage,67.0,3.39,13.46,0.00,0.0,8.39,64.0,0.024,0.32,1.0,1.1,0.04
132,448,92611100,Oatmeal beverage with milk,Grain drinks,Beverage,81.0,1.74,15.49,1.59,0.2,13.97,14.0,0.027,0.22,0.0,0.5,0.03
0,8,11432000,"Yogurt, low fat milk, fruit",Yogurt,Food,89.0,4.66,14.46,1.37,0.1,12.01,12.0,0.046,0.49,1.8,1.2,0.04
133,449,92613010,Cornmeal beverage,Grain drinks,Beverage,83.0,1.70,16.30,1.54,0.2,13.87,14.0,0.039,0.22,0.0,0.5,0.03


**5. Save the model to pickle**

In [ ]:
import pickle
 
pickle.dump(nbrs, open('content_based_knn_model.pkl', 'wb'))
 
model = pickle.load(open('content_based_knn_model.pkl', 'rb'))
distances, indices = model.kneighbors(knn_df)